In [1]:
import numpy as np
from copy import deepcopy

In [2]:
def solve(c: np.array, x: np.array, D: np.array, A: np.array, b: np.array, J_b: list, J_b_ext: list):
    c = deepcopy(c)
    x = deepcopy(x)
    D = deepcopy(D)
    A = deepcopy(A)
    b = deepcopy(b)
    J_b = deepcopy(J_b)
    J_b = deepcopy(J_b_ext)

    while True:
        #1
        A_b = A[:, J_b]
        A_b_inv = np.linalg.inv(A_b)
        c_dash = c + x @ D
        u_dash = -c_dash[J_b] @ A_b_inv
        delta_dash = u_dash @ A + c_dash

        #2
        if (delta_dash >= 0).all():
            return x
        
        #3
        j_0 = np.argmax(delta_dash < 0)

        #4
        l = np.zeros(len(x))
        l[j_0] = 1
        A_b_ext = A[:, J_b_ext]

        H = np.bmat([
            [D[J_b_ext, :][:, J_b_ext], A_b_ext.T],
            [A_b_ext, np.zeros((len(A), len(A)))]
        ])
        H_inv = np.linalg.inv(H)

        b_asterisk = np.concatenate((D[J_b_ext, j_0], A[:, j_0]))
        x_ = np.array(-H_inv @ b_asterisk)[0]
        l[:len(J_b_ext)] = x_[:len(J_b_ext)]
        
        #5
        delta = l @ D @ l
        theta = {}
        theta[j_0] = np.inf if delta == 0 else np.abs(delta_dash[j_0]) / delta
        
        for j in J_b_ext:
            theta[j] = -x[j] / l[j] if l[j] < 0 else np.inf
        
        j_asterisk = min(theta, key=theta.get)
        theta_0 = theta[j_asterisk]

        if theta_0 == np.inf:
            return 'Целевая функция задачи не ограничена снизу на множестве допустимых планом.'
        
        #6
        x = x + theta_0 * l
        if j_asterisk == j_0:
            J_b_ext.append(j_asterisk)
        elif j_asterisk in J_b_ext and j_asterisk not in J_b:
            J_b_ext.remove(j_asterisk)
        elif j_asterisk in J_b:
            third_condition = False
            s = J_b.index(j_asterisk)

            for j_plus in set(J_b_ext).difference(J_b):
                if (A_b_inv @ A[:, j_plus])[s] != 0:
                    third_condition = True
                    J_b[s] = j_plus
                    J_b_ext.remove(j_asterisk)
            # if not third_condition or J_b == J_b_ext:
            if not  third_condition:
                J_b[s] = j_0
                J_b_ext[J_b_ext.index(j_asterisk)] = j_0

In [3]:
x = np.array([2, 3, 0, 0])
J_b = [0, 1]
J_b_ext = [0, 1]
c = np.array([-8, -6, -4, -6])
A = np.array([[1, 0, 2, 1], [0, 1, -1, 2]])
D = np.array([[2, 1, 1, 0], [1, 1, 0, 0], [1, 0, 1, 0], [0, 0, 0, 0]])
b = np.array([2, 3])

solve(c, x, D, A, b, J_b, J_b_ext)

array([1.7, 2.4, 0. , 0.3])

In [4]:
tests = [
    {
        'c': np.array([-8., -6., -4., -6.]),
        'x': np.array([2., 3., 0., 0.]),
        'D': np.array([[2., 1., 1., 0.],
                       [1., 1., 0., 0.],
                       [1., 0., 1., 0.],
                       [0., 0., 0., 0.]]),
        'A': np.array([[1., 0., 2., 1.],
                       [0., 1., -1., 2.]]),
        'b': np.array([2., 3.]),
        'J_b': [0, 1],
        'J_b_ext': [0, 1]
    },
    {
        'c': np.array([0, -1, 0]),
        'x': np.array([0, 2, 1]),
        'D': np.array([[2, -1, 0],
                       [-1, 2, -1],
                       [0, -1, 2]]),
        'A': np.array([[2, 1, 0],
                       [0, 1, 2]]),
        'b': None,
        'J_b': [1, 2],
        'J_b_ext': [1, 2]
    },
    {
        'c': np.array([0, 0, -2]),
        'D': np.array([[1, 0, 0],
                       [0, 1, -1],
                       [0, -1, 2]]),
        'A': np.array([[0, 1, 1],
                       [1, 0, 1]]),
        'x': np.array([2, 4, 0]),
        'b': None,
        'J_b': [0, 1],
        'J_b_ext': [0, 1]
    },
]

In [5]:
# for test in tests:
#     print(solve(test['c'], test['x'], test['D'], test['A'], test['b'], test['J_b'], test['J_b_ext']))

In [6]:
lalala = {
        'c': np.array([0, -1, 0]),
        'x': np.array([0, 2, 1]),
        'D': np.array([[2, -1, 0],
                       [-1, 2, -1],
                       [0, -1, 2]]),
        'A': np.array([[2, 1, 0],
                       [0, 1, 2]]),
        'b': None,
        'Jb': [1, 2],
        'Jb_ast': [1, 2]
    }

In [7]:
print(solve(lalala['c'], lalala['x'], lalala['D'], lalala['A'], lalala['b'], lalala['Jb'], lalala['Jb_ast']))

LinAlgError: Singular matrix